In [75]:
# Implementing freeze-thaw bayesian optimization with two-level Gaussian processes
# A global GP models the asymptotic mean of the learning curves of each HP-config
# Local GPs model the learning curves of each HP-config

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, Kernel, Sum, WhiteKernel
import numpy as np
import matplotlib.pyplot as plt
from ft_testfunction import global_xD, local_xD
import scipy as sc

# Hyper-Hyperparameters of Freeze-Thaw Bayesian Optimization
ALPHA,BETA = 1,0.5
NOISE = 0.1
B_OLD=2 # 10
B_NEW=4 # 3
MAX_EPOCHS=100
EPOCH_STEP=1
N_SAMPLES_MC=10 # number of samples for Monte Carlo integration
N_FANT=1 # 5 # number of observations we fantasize for each point in the basket
N_INIT_CONFIGS=10 # number of random initializations for the optimization
N_INIT_EPOCHS=5 # number of epochs trained for initial configs
INFERRED_MEAN = 0.8 # inferred mean of the global GP
MATER_NU = 2.5 # Matern kernel parameter
EI_N_SAMPLES = 1000 #10000 # number of samples for EI optimization

# Meta-Parameters of the task
OBS_MIN = 0 # minimal loss value
OBS_MAX = 1 # maximal loss value


In [76]:
'''
Equation 3: Expected Improvement-formula
Equation 4: Entropy Search-formula
Equation 5&6: Exponential Decay-kernel for Gaussian Process
Equation 12,13,17 & 18: Posterior distribution of global Gaussian Process
Equation 14 & 19: Posterior predictive distribution of global Gaussian Process
Equation 15 & 20: Posterior predictive distribution of a local Gaussian Process with existing oberservations of this HP-config
Equation 16 & 21: Posterior predicitve distribution of a local Gaussian Process without existing oberservations of this HP-config
'''


'\nEquation 3: Expected Improvement-formula\nEquation 4: Entropy Search-formula\nEquation 5&6: Exponential Decay-kernel for Gaussian Process\nEquation 12,13,17 & 18: Posterior distribution of global Gaussian Process\nEquation 14 & 19: Posterior predictive distribution of global Gaussian Process\nEquation 15 & 20: Posterior predictive distribution of a local Gaussian Process with existing oberservations of this HP-config\nEquation 16 & 21: Posterior predicitve distribution of a local Gaussian Process without existing oberservations of this HP-config\n'

In [77]:
# Definition of exponential-decay kernel for local GPs
class ExponentialDecayNoiseKernel(Kernel):
    def __init__(self, alpha=1.0,beta=0.5,noise=0.1):
        self.beta=beta
        self.alpha=alpha
        self.noise = noise
    def __call__(self, X, Y=None,eval_gradient=False):
        if Y is None:
            Y = X
        # if eval_gradient:
        #     return (self.beta**self.alpha)/((X.flatten()[:,None]+Y.flatten()[None,:])+self.beta)**self.alpha,np.identity(X.shape[0])
        X=np.array(X)
        Y=np.array(Y)
        return ((self.beta**self.alpha)/((X[None,:]+Y[:,None])+self.beta)**self.alpha + self.noise*np.where(X[None,:]-Y[:,None] == 0, 1, 0)).T
    def diag(self, X):
        return np.diag(self(X))
    def is_stationary(self):
        return False

In [78]:
# Start by observing N_INIT_CONFIGS random configurations for N_INIT_EPOCHS epoch each
bounds={'HP1':(1.,5.),'HP2':(1.,5.)}
observed_configs_dicts={}
observed_configs_list=[]
for i in range(N_INIT_CONFIGS):
    new_config=np.empty(0)
    for key in bounds.keys():
        new_config=np.append(new_config,np.round(np.random.uniform(bounds[key][0],bounds[key][1]),2))
    # Observe the new configuration for N_INIT_EPOCHS epochs
    f_space = np.linspace(1,N_INIT_EPOCHS,N_INIT_EPOCHS)
    experimental_data=local_xD(np.array(new_config),f_space,noise=0.01)
    observed_configs_dicts['_'.join([str(config) for config in new_config])]=(f_space,experimental_data)


    observed_configs_list.append(new_config)
observed_configs_list=np.array(observed_configs_list)



print(observed_configs_dicts)
print(observed_configs_list)



{'1.11_1.34': (array([1., 2., 3., 4., 5.]), array([0.95, 0.91, 0.86, 0.81, 0.77])), '4.23_2.82': (array([1., 2., 3., 4., 5.]), array([0.95, 0.87, 0.83, 0.78, 0.74])), '4.99_1.05': (array([1., 2., 3., 4., 5.]), array([0.98, 0.94, 0.93, 0.9 , 0.87])), '4.5_1.74': (array([1., 2., 3., 4., 5.]), array([0.95, 0.91, 0.87, 0.81, 0.8 ])), '2.96_3.56': (array([1., 2., 3., 4., 5.]), array([0.93, 0.84, 0.81, 0.76, 0.7 ])), '3.04_4.69': (array([1., 2., 3., 4., 5.]), array([0.92, 0.85, 0.81, 0.74, 0.69])), '1.79_4.4': (array([1., 2., 3., 4., 5.]), array([0.93, 0.88, 0.83, 0.79, 0.75])), '4.95_1.07': (array([1., 2., 3., 4., 5.]), array([0.98, 0.95, 0.94, 0.92, 0.89])), '2.01_3.02': (array([1., 2., 3., 4., 5.]), array([0.95, 0.88, 0.81, 0.77, 0.74])), '4.74_4.72': (array([1., 2., 3., 4., 5.]), array([0.96, 0.92, 0.89, 0.86, 0.84]))}
[[1.11 1.34]
 [4.23 2.82]
 [4.99 1.05]
 [4.5  1.74]
 [2.96 3.56]
 [3.04 4.69]
 [1.79 4.4 ]
 [4.95 1.07]
 [2.01 3.02]
 [4.74 4.72]]


In [79]:
# from botorch.models import SingleTaskGP
# from gpytorch.kernels import MaternKernel, ScaleKernel
# from gpytorch.means import ConstantMean
# import torch
#MaternKernel(nu=2.5)
# print(torch.from_numpy(observed_configs_list))


# Define the kernel for the global GP
kernel_global = Matern(nu=MATER_NU)
kernel_local = ExponentialDecayNoiseKernel(alpha=ALPHA,beta=BETA,noise=NOISE)

def compute_k_x(x,new_x,kernel):
    k_x = kernel_global(x)
    k_x_star = kernel_global(x,new_x)
    k_star_star = kernel_global(new_x)
    return k_x,k_x_star,k_star_star

def compute_k_t_n(x,new_x,kernel):
    k_t=kernel(x)
    k_t_star=kernel(x,new_x)
    k_star_star=kernel(new_x)
    return k_t,k_t_star,k_star_star

def compute_k_t(x,x_dict):
    k_ts=[]
    for x_n in x:
        k_ts.append(kernel_local(x_dict['_'.join([str(c) for c in x_n])][0]))
    return sc.linalg.block_diag(*k_ts)

def compute_o(config_list,config_dict):
    o=[]
    for config in config_list:
        observations=config_dict['_'.join([str(c) for c in config])][1]
        o=sc.linalg.block_diag(*([o,np.ones((len(observations),1))]))
    return o[1:]

def compute_lambda(o,k_t_inv):
    return o.T@k_t_inv@o

def compute_gamma(o,k_t_inv,y,m):
    return o.T@k_t_inv@(y-o@m)

def constant_mean(x):
    return INFERRED_MEAN*np.ones(x.shape[0])

def compute_y_vector(config_list,config_dict):
    y_vec=np.empty(0)
    for config in config_list:
        observations=config_dict['_'.join([str(c) for c in config])][1]
        y_vec=np.append(y_vec,observations)
    return y_vec

def compute_c(k_x_inv,lambd):
    return np.linalg.inv(k_x_inv+lambd)

def compute_omega(k_t_n_star,k_t_n_inv):
    obs_steps=k_t_n_inv.shape[0]
    predict_steps=k_t_n_star.shape[1]
    return np.ones(predict_steps)-k_t_n_star.T@k_t_n_inv@np.ones(obs_steps)

def compute_mu(m,c,gamma):
    return (m+c@gamma)

# Equation 14/19:
def compute_mu_x_star(m,k_x_star,k_x_inv,mu,means_vec):
    return m+k_x_star.T@k_x_inv@(mu-means_vec)
def compute_sigma_x_star_star(k_x_star_star,k_x_star,k_x,lambd_inv):
    return k_x_star_star-k_x_star.T@np.linalg.inv(k_x+lambd_inv)@k_x_star

# Equation 16/21:
def compute_sigma_n_star_new(k_t_star_star,sigma_star_star):
    return k_t_star_star-sigma_star_star

# Equation 15/20:
def compute_mu_n_star_ex(k_t_n_star,k_t_n_inv,y_n,omega_n,mu_n):
    return k_t_n_star.T@k_t_n_inv@y_n+(omega_n*mu_n)
def compute_sigma_n_star_ex(k_t_n_star_star,k_t_n_star,k_t_n_inv,omega_n,c_nn):
    return k_t_n_star_star-k_t_n_star.T@k_t_n_inv@k_t_n_star+omega_n@(c_nn*omega_n.T)

In [80]:
new_x = np.array([[observed_configs_list[0,0],2],[3,2],[1,2]]) # Placeholer
print(observed_configs_list)
print(new_x)
k_x,k_x_star,k_x_star_star = compute_k_x(observed_configs_list,new_x,kernel_global)
k_x_inv = np.linalg.inv(k_x)
# print(f"Kx:\n{k_x}\n Kx*:\n{k_x_star}\n Kx**:\n{k_x_star_star}")

new_x_n=np.array([100]) # Placeholer
curve=observed_configs_dicts["_".join([str(x) for x in observed_configs_list[0]])]
print(observed_configs_list[0])
# print(curve)
# print(new_x_n)
k_t_n,k_t_n_star,k_t_n_star_star=compute_k_t_n(curve[0],new_x_n,kernel_local)
k_t_n_inv=np.linalg.inv(k_t_n)
# print(f"Ktn:\n{k_t_n}\n Ktn*:\n{k_t_n_star}\n Ktn**:\n{k_t_n_star_star}")

k_t=compute_k_t(observed_configs_list,observed_configs_dicts) # Placeholer
k_t_inv=np.linalg.inv(k_t)
# print(f"Kt:\n {k_t}")

o = compute_o(observed_configs_list,observed_configs_dicts)
# print(f"O:\n{o}")

lambd = compute_lambda(o,k_t_inv)
lambd_inv = np.linalg.inv(lambd)
# print(f"Lambda:\n{lambd}")

means_vec = constant_mean(observed_configs_list)
# print(f"Means:\n{means_vec}")

y_vec = compute_y_vector(observed_configs_list,observed_configs_dicts)
# print(f"Y:\n{y_vec}")

gamma = compute_gamma(o,k_t_inv,y_vec,means_vec)
# print(f"Gamma:\n{gamma}")

c = compute_c(k_x_inv,lambd)
# print(f"C:\n{c}")

mu_global = compute_mu(means_vec,c,gamma)
# print(f"Mu:\n{mu_global}")

omega_n = compute_omega(k_t_n_star,k_t_n_inv)
# print(f"Omega n:\n{omega_n}")

mu = compute_mu_x_star(constant_mean(new_x),k_x_star,k_x_inv,mu_global,means_vec)
print(f"μx*:\n{mu}")

var = compute_sigma_x_star_star(k_x_star_star,k_x_star,k_x,lambd_inv)
print(f"Σx**:\n{var}")

mu_n_star_ex = compute_mu_n_star_ex(k_t_n_star,k_t_n_inv,curve[1],omega_n,mu_global[0])
print(f"μn* (existing):\n{mu_n_star_ex}")

sigma_n_star_ex = compute_sigma_n_star_ex(k_t_n_star_star,k_t_n_star,k_t_n_inv,omega_n,c[0,0])
print(f"Σn* (existing):\n{sigma_n_star_ex}")

[[1.11 1.34]
 [4.23 2.82]
 [4.99 1.05]
 [4.5  1.74]
 [2.96 3.56]
 [3.04 4.69]
 [1.79 4.4 ]
 [4.95 1.07]
 [2.01 3.02]
 [4.74 4.72]]
[[1.11 2.  ]
 [3.   2.  ]
 [1.   2.  ]]
[1.11 1.34]
μx*:
[0.8057105  0.78969852 0.80628098]
Σx**:
[[ 0.45448727  0.01268031  0.45542951]
 [ 0.01268031  0.79849291 -0.00256883]
 [ 0.45542951 -0.00256883  0.47529131]]
μn* (existing):
[0.81317427]
Σn* (existing):
[[0.17646856]]


In [81]:
def compute_EI_at_x(mu,var,best_mu):
    z=(best_mu-mu)/np.sqrt(var)
    return np.sqrt(var)*(z*sc.stats.norm.cdf(z))+sc.stats.norm.pdf(z)


In [82]:

'''
Our Bayesian optimization strategy proceeds by maintaining a basket of B = Bold + Bnew candidate
models. Bold represents some number of models that have already been trained to some degree, while Bnew
represents some number of brand new models. In practice, we set Bold = 10 and Bnew = 3. The entire
basket is chosen using models with the maximum EI at the asymptote, which is computed using Equations
19 and 3. Each round, after a new observation has been collected, the basket is re-built using possibly
different models. This step is essentially standard Bayesian optimization using EI.
'''

# Fill the basket with configs
basket_new=np.empty((0,len(bounds.keys())))
basket_old=np.empty((0,len(bounds.keys())))

# Get the best yet observed configuration
best_observation = np.min(np.concatenate([observed_configs_dicts['_'.join([str(c) for c in config])][1] for config in observed_configs_list]))
# print(f"Best observation: {best_observation}")

# Calculate EI for many configs to find the best ones
while not (len(basket_new)==B_NEW) and not (len(basket_old)==B_OLD):

    # Sample N_EI_SAMPLES new configurations
    ei_configs = []
    for i in range(EI_N_SAMPLES):
        while True:
            new_config=np.empty(0)
            for key in bounds.keys():
                new_config=np.append(new_config,np.round(np.random.uniform(bounds[key][0],bounds[key][1]),2))
            if not new_config in observed_configs_list:
                break
        ei_configs.append(new_config)
    if len(basket_old)<B_OLD:
        ei_configs = np.concatenate([ei_configs,observed_configs_list])
    ei_configs=np.array(ei_configs)
    # print(ei_configs)

    # Calculate the mean and variance at the asymptote for each config using equation 19
    k_x,k_x_star,k_x_star_star = compute_k_x(observed_configs_list,ei_configs,kernel_global)
    k_x_inv = np.linalg.inv(k_x)
    k_t=compute_k_t(observed_configs_list,observed_configs_dicts)
    k_t_inv=np.linalg.inv(k_t)
    means_vec = constant_mean(observed_configs_list)
    o=compute_o(observed_configs_list,observed_configs_dicts)
    lambd = compute_lambda(o,k_t_inv)
    c=compute_c(k_x_inv,lambd)
    y_vec = compute_y_vector(observed_configs_list,observed_configs_dicts)
    gamma = compute_gamma(o,k_t_inv,y_vec,means_vec)
    mu_global = compute_mu(means_vec,c,gamma)
    mu = compute_mu_x_star(constant_mean(ei_configs),k_x_star,k_x_inv,mu_global,means_vec)
    # print(f"μx*:\n{mu}")
    cov = compute_sigma_x_star_star(k_x_star_star,k_x_star,k_x,lambd_inv)
    var=np.diag(cov)
    # print(f"Σx**:\n{var}")

    # Calculate the EI scores for each config using equation 3
    ei_scores=compute_EI_at_x(mu,var,best_observation)
    sort_indices = np.argsort(ei_scores)[::-1]
    ei_configs_ranked = ei_configs[sort_indices]
    # print(ei_configs_ranked)


    # Greedily choose the best HP-config using Equation 19 & 3 until B_OLD existing configs and B_NEW new configs are found

    for sampled_EI_config in ei_configs_ranked:
        # Sample another config from EI and try to add it to the basket
        # If it is already in the basket, or the basket is full, skip it
        if not np.any(np.all(np.isin(observed_configs_list,sampled_EI_config),axis=1)) and (basket_new.shape[0]==0 or B_NEW>basket_new.shape[0] and not np.any(np.all(np.isin(basket_new,sampled_EI_config),axis=1))):
            # print(f"Adding new config to basket_new: {sampled_EI_config}")
            basket_new=np.vstack([basket_new,sampled_EI_config])
        elif np.any(np.all(np.isin(observed_configs_list,sampled_EI_config),axis=1)) and (basket_old.shape[0]==0 or B_OLD>basket_old.shape[0] and not np.any(np.all(np.isin(basket_old,sampled_EI_config),axis=1))):
            # print(f"Adding new config to basket_old: {sampled_EI_config}")
            basket_old=np.vstack([basket_old,sampled_EI_config])


print(f"New basket:\n{basket_new}")
print(f"Old basket:\n{basket_old}")

New basket:
[[3.   4.25]
 [2.98 4.34]
 [2.86 4.43]
 [3.05 4.28]]
Old basket:
[[3.04 4.69]
 [2.96 3.56]]


In [83]:
# Compute p_min over the basket using Monte Carlo simulation and Equation 19

# p_min is the posterior predictive distribution over the minimum of the global GP
# We will use it's entropy to observe configs with results that make the p_min distribution spikier

# Monte Carlo simulation
# Using the global GP, we create a discrete grid of the N_SAMPLES_MC highest EI configs
# From the grid, compute P_min and the entropy of P_min, H(P_min)

h_p_min=1
a=np.zeros(basket_new.shape[0]+basket_old.shape[0])


In [84]:
# For each config in the basket, N_FANT times fantasize an observation and recompute the information gain from it, collecting it in a
for k,config in enumerate(basket_new):
    for f_n in range(N_FANT):
        # Fantasize an observation using Equation 21
        # TODO fantasize an observation
        print(f"Chosen new config: {config}")
        
        

        # Compute p_min_y, the new minimum distribution, including the fantasized observation y, again using Monte Carlo and Equation 19
        # TODO compute p_min_y

        # Compute the new entropy of p_min_y, H(P_min_y)
        # TODO compute H(P_min_y)
        h_p_min_y=0.5
        # 
        a[k]+=(h_p_min_y-h_p_min)/N_FANT

for k,config in enumerate(basket_old):
    for f_n in range(N_FANT):
        # Fantasize an observation using Equation 20
        # TODO fantasize an observation

        # Compute p_min_y, the new minimum distribution, including the fantasized observation y, again using Monte Carlo and Equation 19
        # TODO compute p_min_y

        # Compute the new entropy of p_min_y, H(P_min_y)
        # TODO compute H(P_min_y)
        h_p_min_y=0

        a[k+B_NEW]+=(h_p_min_y-h_p_min)/N_FANT


# Select the config with the highest information gain
best_config=(basket_new+basket_old)[np.argmax(a)]
print(best_config)


Chosen new config: [3.   4.25]
Chosen new config: [2.98 4.34]
Chosen new config: [2.86 4.43]
Chosen new config: [3.05 4.28]


ValueError: operands could not be broadcast together with shapes (4,2) (2,2) 